# 1. Carregar biblioteca e funções

In [ ]:
# Bibiotecas padrão
import pandas as pd
import numpy as np

## Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp

# Escrever as fórmulas dos modelos
import patsy as pt

# Bibioteca estatística
import statsmodels.api as sm

# Fator de inflação da variância
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Bibliotecas do scikitlearn para calcular as métricas e fazer a regressão regularizada
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.linear_model import Ridge, RidgeCV

In [ ]:
def desc_null(df, percentiles=False):
  """Função para descrição de dados, incluindo colunas de contagem de nulos e porcentagem de nulos em cada variável

  Args:
      df (pd.DataFrame): base de dados para descrição
      percentiles (bool, optional): exibição dos percentis [0.05, 0.25, 0.5, 0.75, 0.9, 0.99]. Defaults to False.

  Returns:
      des(pd.DataFrame): descrição dos dados com a quantidade e porcentagem de nulos por feature
  """  
  import pandas as pd

  if percentiles is True:
    des = df.describe(datetime_is_numeric=True,
                      percentiles=[0.05, 0.25, 0.5, 0.75, 0.9, 0.99]).T
    des["nullCount"] = df.shape[0] - des["count"]
    des["null%"] = (des["nullCount"] / df.shape[0]) * 100

  else:
    des = df.describe(include='all', 
                      datetime_is_numeric=True).T
    des['nullCount'] = df.shape[0] - des['count']
    des['null%'] = (des['nullCount'] / df.shape[0]) * 100
    
  return des

In [ ]:
def cat_filter(df, columns, entries, drop_first=True):
  """Filtro de categorias, gera uma nova categoria Others com os menos representativos, retornando nova base com os dados dummyficados

  Args:
      df (pd.DataFrame): base de dados
      columns (list): lista com as colunas com dados categóricos
      entries (int): quantidade mínima de entradas significativas de categorias
      drop_first(bool, optional): remoção da primeira categoria para evitar multicolinearidade. Defaults to True.

  Returns:
      df(pd.DataFrame): base de dados com a dummyficação dos valores categóricos
  """  
  import pandas as pd
  
  filter = {}
  for col in columns:
    cat = df[col].value_counts().sort_values(ascending=True)
    filter = {x: 'Other' for x in cat[cat < entries].index}
    
  df.loc[:, col] = df[col].replace(filter).copy()
  df = pd.get_dummies(df, columns=columns, drop_first=drop_first)
  
  return df

In [ ]:
def histplot(data, title=None, xlabel=None, ylabel=None, x=None, y=None, ax=False, nrows=1, ncols=1, figsize=(10, 5), bins=10):
  """Gráfico de frequência pra visualização de dados numéricos e categóricos

  Args:
      data (pd.DataFrame): base de dados
      title (str, optional): título do gráfico, opcional. Defaults to None.
      xlabel (str, optional): título do eixo x, opcional. Defaults to None.
      ylabel (str, optional): título do eixo y, opcional. Defaults to None.
      x (str, optional): feature da base de dados para o eixo x. Defaults to None.
      y (str, optional): feature da base de dados para o eixo y. Defaults to None.
      ax (bool, optional): determina se a impressão conterá múltiplos gráficos. Defaults to False.
      nrows (int, optional): quantidade de linhas. Defaults to 1.
      ncols (int, optional): quantidade de colunas. Defaults to 1.
      figsize (tuple, optional): tamanho da impressão. Defaults to (10, 5).
      bins (int, optional): quantidade de bins. Defaults to 10.

  Returns:
      sns.histplot: impressão do gráfico
  """  
  import matplotlib.pyplot as plt
  import seaborn as sns
  
  if ax is True:
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    c = 0
    if len(x) < (nrows * ncols):      
      for i in range(nrows):
        for j in range(ncols):
          if i == nrows-1 and j == ncols-1:
            fig.delaxes(axes[nrows-1,ncols-1])
          else:
            sns.histplot(data=data, x=x[c], y=y, bins=bins, ax=axes[i,j]).set_ylabel(ylabel)
            c += 1
      
    elif nrows > 1 and ncols > 1:      
      for i in range(nrows):
        for j in range(ncols):
          sns.histplot(data=data, x=x[c], y=y, bins=bins, ax=axes[i,j]).set_ylabel(ylabel)
          c += 1
    elif nrows == 1:
      for j in range(ncols):
        sns.histplot(data=data, x=x[c], y=y, bins=bins, ax=axes[j]).set_ylabel(ylabel)
        c += 1
    elif ncols == 1:
      for i in range(nrows):
        sns.histplot(data=data, x=x[c], y=y, bins=bins, ax=axes[i]).set_ylabel(ylabel)
        c += 1
    if title is not None:
      fig.suptitle(title)
  
  else:
    plt.figure(figsize=figsize)
    sns.histplot(data=data, x=x, y=y, bins=bins)
    
    if title is not None:
      plt.title(title)
    if xlabel is not None:
      plt.xlabel(xlabel)
    if ylabel is not None:
      plt.ylabel(ylabel)
      
  return plt.show()

In [ ]:
def corrplot(data):
    """Gráfico de correlação com paleta de cores do azul ao vermelho e anotações para melhor entendimento dos dados

    Args:
        data (pd.DataFrame): base de dados para realizar a correlação

    Returns:
        sns.heatmap: impressão do gráfico
    """    
    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(10, 7))

    sns.heatmap(
        data=data.corr(),
        cmap=sns.diverging_palette(230, 20, as_cmap=True),
        vmin=-1,
        vmax=1,
        annot=True,
    )

    return plt.show()

In [ ]:
def norm_test(df):
    """Realiza teste de normalidade de Kolmorogov no resíduo da previsão

    Args:
        df (pd.Series): resíduo da previsão

    Returns:
        data(pd.DataFrame): tabela com as informações estatísticas e P-Value do resíduo de previsão
    """ 
    import scipy as sp
    import pandas as pd

    kStat, kPvalue = sp.stats.kstest(df, 'norm')
    data = pd.DataFrame({'kStat': kStat,
                         'P-Value': kPvalue},
                        index=['Kolmogrov']).round(decimals=5)

    return data

In [ ]:
def feat_scale(x):
    """Escalonar as variáveis numéricas da base de dados

    Args:
        x (pd.DataFrame): base de dados com as variáveis numéricas

    Returns:
        xnorm(float): valores escalonados
    """    
    import numpy as np

    # Calcular a média da variável
    mean = np.mean(x, axis=0)

    # Calcular o desvio padrão amostral da variável
    sigma = np.std(x, axis=0, ddof=1)

    # Realizar o escalonamento
    xnorm = (x - mean) / sigma

    return xnorm

In [ ]:
def ols_formula(df, dependent_var, *excluded_rows):
    """Fórmula para seleção das colunas para gerar a matriz dos dados

    Args:
        df (pd.DataFrame): base de dados
        dependent_var (str): variável resposta
        * excluded_rows (str): variáveis a serem excluídas

    Returns:
        dependent_var(list): relação das colunas para criação da matriz
    """    

    # Listar o nome das colunas do dataframe
    cols = list(df.columns)

    # Remover a variável dependente
    cols.remove(dependent_var)

    # Remover as variáveis excluídas
    for col in excluded_rows:
        cols.remove(col)

    # Retornar a fórmula
    return dependent_var + " ~ " + " + ".join(cols)

In [ ]:
def fit_model(y, x, type='OLS', alpha=1.0):
  
  """Ajuste do modelo para os métodos OLS e Ridge

  Args:
      y (pd.DataFrame): variável resposta
      x (pd.DataFrame): regressores
      type (str, optional): tipo de modelo OLS ou Ridge. Defaults to 'OLS'.
      alpha (float, optional): alpha do modelo Ridge para ajuste. Defaults to 1.0.

  Returns:
      regfit(): modelo ajustado
  """  
  import statsmodels.api as sm
  from sklearn.linear_model import Ridge
  
  if type == 'OLS':
    reg = sm.OLS(y, x)
    regfit = reg.fit()
  elif type == 'Ridge':
    regfit = Ridge(alpha=alpha)
    regfit.fit(x, y)

  return regfit

In [ ]:
def vif_view(x, intercept=False):
    """Fator de inflação da variância

    Args:
            x (pd.DataFrame): base de dados dos regressores
            intercept (bool, optional): considerar ou não a coluna Intercept. Defaults to False.

    Returns:
            vif(pd.DataFrame): tabela com as variáveis e seus respectivos VIFs
    """    
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor


    if intercept is True:
            vif = pd.DataFrame({'Variáveis': x.columns[1:],
                                'VIF': [variance_inflation_factor(x.values, i + 1)
                                        for i in range(len(x.columns[1:]))]
                            })
    else:
            vif = pd.DataFrame({'Variáveis': x.columns,
                                'VIF': [variance_inflation_factor(x.values, i + 1)
                                        for i in range(len(x.columns))]
                            })
            
    return vif

In [ ]:
def predict_table(y, fit, columns, residual=False):
    """Tabela de previsão e cálculo de resíduo do modelo

    Args:
        y (pd.DataFrame): variável resposta
        fit (.predict): predição do modelo ajustado dos regressores
        columns (list): nome das colunas Real e Previsto
        residual (bool, optional): cálculo do residual dos regressores. Defaults to False.

    Returns:
        pred(pd.DataFrame): base com predição do modelo ajustado
    """    
    import pandas as pd
    
    pred = pd.concat([y, fit], axis=1)
    pred.columns = columns

    if residual is True:
        pred['Res'] = pred[columns[0]] - pred[columns[1]]

    return pred

In [ ]:
def adj_quality(pred):
    """Qualidade de ajuste do modelo com R² e RMSE

    Args:
        pred (pd.DataFrame): tabela de previsão do modelo

    Returns:
        adj(pd.DataFrame): cálculo de R² e RMSE do modelo
    """    
    import pandas as pd
    import numpy as np
    from sklearn import metrics

    adj = pd.DataFrame({
        'R²': metrics.r2_score(pred.loc[:, pred.columns[0]], 
                               pred.loc[:, pred.columns[1]]),
        'RMSE': np.sqrt(metrics.mean_squared_error(pred.loc[:, pred.columns[0]], 
                                                   pred.loc[:, pred.columns[1]]))
    }, index=[0])

    return adj

In [ ]:
def funcao_perda(x, y, beta):
    """Cria a função de perda para o gradiente descendente

    Args:
        x (pd.DataFrame): matriz dos regressores
        y (pd.Series): série com a resposta
        beta (np.array): série dos parâmetros

    Returns:
        perda: cálculo da perda
    """    
    import numpy as np

    m, n = x.shape

    # Previsão da resposta: produto escalar entre os vetores de parâmetros e o dataframe com os regressores - produto escalar entre x e beta
    pred = x.dot(beta)

    # Calcular os erros de previsão: é a diferença entre o valor real da variável y e o valor previsto pelo modelo - resíduo
    res = np.subtract(pred, y.squeeze())

    # Calcular o quadrado dos erros
    sqrRes = np.square(res)

    # Calcular a perda
    perda = 1 / (2 * m) * np.sum(sqrRes)

    return perda

In [ ]:
def grad_des(x, y, alpha, itera):
    """Cálculo do gradiente descendente

    Args:
        x (pd.DataFrame): matriz dos regressores
        y (pd.Series): série da resposta
        alpha (float): taxa de aprendizagem escalar do modelo
        itera (int): número de iterações do algoritmo

    Returns:
        beta(pd.Series): série com os valores finais dos parâmetros
        hPerda(pd.Series): série com o histórico de perdas
    """    
    import numpy as np

    # Define o número de linhas e o de regressores
    m, n = x.shape
    
    # Inicializa os valores dos betas
    beta = np.zeros(n)

    # Inicializa a série do histórico de perdas: ajuda a encontrar os melhores valores de alpha para alcançar uma melhor convergência
    hPerda = np.zeros(itera)

    for i in range(itera):

        # Calcula as previsões com os valores atuais dos parâmetros
        pred = x.dot(beta)

        # Calcula os resíduos de previsão: previsão - resposta
        res = np.subtract(pred, y.squeeze())

        # Calcula o incremento/decremento valor dos betas - derivada parcial da função perda:
        # quanto maior o alpha, maior o passo que o algoritmo dará para o próximo beta
        sumDelta = (alpha / m) * x.transpose().dot(res)

        # Atualizar os valores dos betas do modelo
        beta = beta - sumDelta

        # Calcula a nova perda com os novos valores dos betas
        hPerda[i] = funcao_perda(x, y, beta)

    return beta, hPerda

# 2. Carregar os dados

In [ ]:
file = 
df = pd.read_csv(file)

In [ ]:
df.head()

# 3. Análise Exproratória dos dados

## 3.1. Separação das variáveis e tratamento de nulos

In [ ]:
desc_null(df)

### 3.1.1. Variáveis categóricas

In [ ]:
cat = []

dfCat = df.loc[:, c]
dfCat.head()

### 3.1.2. Variáveis numéricas

In [ ]:
num = []
resp = []
# num = list(df.select_dtypes(include=''))
# num.remove(resp)

In [ ]:
dfNum = df.loc[:, num]
dfNum.head()

In [ ]:
desc_null(dfNum, percentiles=True)

### 3.1.3. Variável resposta

In [ ]:
dfResp = df.loc[:, resp]
dfResp.head()

## 3.2. Distribuição das variáveis

In [ ]:
histplot(data=dfCat, 
         title='Distribuição das variáveis categóricas', 
         ylabel='Frequência',
         x=cat, ax=True,
         ncols=2, nrows=,
         bins=15,
         figsize=(10,5))

In [ ]:
histplot(data=dfNum, 
         title='Distribuição das variáveis numéricas', 
         ylabel='Frequência',
         x=num, ax=True,
         ncols=2, nrows=,
         bins=15,
         figsize=(10,5))